In [11]:
!pip install pyspark
import pyspark

In [12]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *


In [13]:
spark = SparkSession.builder.getOrCreate()

In [15]:
from google.colab import files
uploaded = files.upload()

Saving titanic.csv to titanic.csv


In [16]:
df = spark.read.csv("titanic.csv", header=True, inferSchema=True)
df.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|gender| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| NULL|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| NULL|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| NULL|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+------

In [17]:
null_cabin = df.filter(df.Cabin.isNull()).groupBy("gender").count()
print("Males/Females with null Cabin:")
null_cabin.show()

Males/Females with null Cabin:
+------+-----+
|gender|count|
+------+-----+
|female|  217|
|  male|  470|
+------+-----+



In [18]:
avg_age = df.select(avg("Age")).first()[0]
print(f"Average age: {avg_age:.1f}")


Average age: 29.7


In [19]:
df = df.fillna({"Age": avg_age})

In [20]:
df.write.csv("titanic_processed", header=True)

In [21]:
survivors = df.groupBy("Survived").count()
print("Survivors vs Non-survivors:")
survivors.show()

Survivors vs Non-survivors:
+--------+-----+
|Survived|count|
+--------+-----+
|       1|  342|
|       0|  549|
+--------+-----+



In [22]:
ports = df.groupBy("Embarked").count().orderBy("count", ascending=False).limit(5)
print("Top embarkation ports:")
ports.show()


Top embarkation ports:
+--------+-----+
|Embarked|count|
+--------+-----+
|       S|  644|
|       C|  168|
|       Q|   77|
|    NULL|    2|
+--------+-----+



In [23]:
class_survival = df.groupBy("Pclass").agg(
    (sum("Survived") / count("*") * 100).alias("survival_rate")
)
print("Survival rate by class:")
class_survival.show()


Survival rate by class:
+------+------------------+
|Pclass|     survival_rate|
+------+------------------+
|     1| 62.96296296296296|
|     3|24.236252545824847|
|     2| 47.28260869565217|
+------+------------------+



In [24]:
fare_stats = df.select(
    max("Fare").alias("max_fare"),
    min("Fare").alias("min_fare"),
    avg("Fare").alias("avg_fare")
)
print("Fare statistics:")
fare_stats.show()


Fare statistics:
+--------+--------+----------------+
|max_fare|min_fare|        avg_fare|
+--------+--------+----------------+
|512.3292|     0.0|32.2042079685746|
+--------+--------+----------------+



In [25]:
age_groups = df.withColumn("age_group",
    when(col("Age") <= 18, "0-18")
    .when(col("Age") <= 35, "19-35")
    .when(col("Age") <= 60, "36-60")
    .otherwise("60+")
).groupBy("age_group").count()
print("Passengers by age group:")
age_groups.show()


Passengers by age group:
+---------+-----+
|age_group|count|
+---------+-----+
|    19-35|  535|
|      60+|   22|
|    36-60|  195|
|     0-18|  139|
+---------+-----+



In [26]:
print("First 10 rows:")
df.show(10)


First 10 rows:
+-----------+--------+------+--------------------+------+-----------------+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|gender|              Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+-----------------+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|             22.0|    1|    0|       A/5 21171|   7.25| NULL|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|             38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|             26.0|    0|    0|STON/O2. 3101282|  7.925| NULL|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|             35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|             35.0|    0|    0|  

In [28]:
spark.stop()

In [29]:
import os


In [30]:
os.makedirs("titanic_lab", exist_ok=True)
print("Created titanic_lab directory")


Created titanic_lab directory


In [31]:
import shutil
shutil.copy("titanic.csv", "titanic_lab/")
print("Copied file to titanic_lab")

Copied file to titanic_lab


In [32]:
os.chmod("titanic_lab/titanic.csv", 0o777)
print("Changed permissions to 777")


Changed permissions to 777


In [33]:
print("First 20 lines:")
with open("titanic_lab/titanic.csv", "r") as f:
    for i, line in enumerate(f):
        if i >= 20:
            break
        print(line.strip())

First 20 lines:
PassengerId,Survived,Pclass,Name,gender,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.25,,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.925,,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1,C123,S
5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.05,,S
6,0,3,"Moran, Mr. James",male,,0,0,330877,8.4583,,Q
7,0,1,"McCarthy, Mr. Timothy J",male,54,0,0,17463,51.8625,E46,S
8,0,3,"Palsson, Master. Gosta Leonard",male,2,3,1,349909,21.075,,S
9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27,0,2,347742,11.1333,,S
10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14,1,0,237736,30.0708,,C
11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4,1,1,PP 9549,16.7,G6,S
12,1,1,"Bonnell, Miss. Elizabeth",female,58,0,0,113783,26.55,C103,S
13,0,3,"Saundercock, Mr. William He

In [34]:
os.makedirs("titanic_results", exist_ok=True)
shutil.move("titanic_processed", "titanic_results/")
print("Moved processed output to titanic_results")

Moved processed output to titanic_results
